## Clean recipe data

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from IPython.display import display, Image, HTML
import re
from fuzzywuzzy import fuzz, process
from collections import defaultdict
from fractions import Fraction


user_recipe_df = pd.read_csv("../raw/recipes.csv")

#displays all columns of df instead of truncating
pd.set_option('display.max_columns', None)

print("First few rows: \n")
display(user_recipe_df.head())

## remove duplicates 
#### removing nulls makes shape of recipe df go from 1048543 to 44 (prob cus not all recipes have associated images); omitting####
user_recipe_df = user_recipe_df.drop_duplicates()

# print(user_recipe_df.shape)

## check for and remove anomolies only if using user_food_df
def anomaly_detection(df, columns):
    for column in columns:
        row = df[column]
        Q1 = row.quantile(0.25)
        Q3 = row.quantile(0.75)
        IQR = Q3 - Q1

        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR
        
        anomalies = df[(row < lower_bound) | (row > upper_bound)]
        #print(f'\n\nNumber of anomalies in {column}: ', len(anomalies))

        df_anomaly_free = df[(row >= lower_bound) & (row <= upper_bound)]
        #print(f'\n\nNumber of anomaly-free in {column}: ', len(df), '\n\n')
    
    return df_anomaly_free

columns_to_check = ['RecipeId', 'Calories', 'FatContent', 'SaturatedFatContent', 'CholesterolContent', 
                    'SodiumContent', 'CarbohydrateContent', 'FiberContent', 'SugarContent', 'ProteinContent', 
                    'RecipeServings']


clean_recipe_df = anomaly_detection(user_recipe_df, columns_to_check)
#gets rid of setting with copy warning
clean_recipe_df = clean_recipe_df.copy()


C:\Users\james\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')
C:\Users\james\AppData\Local\Temp\ipykernel_5960\2022180778.py:13: DtypeWarning: Columns (1,2,4,5,6,7,8,9,10,11,12,13,14,27,28) have mixed types. Specify dtype option on import or set low_memory=False.
  user_recipe_df = pd.read_csv("../raw/recipes.csv")


First few rows: 



,RecipeId,Barcode,Name,AuthorId,AuthorName,CookTime,PrepTime,TotalTime,DatePublished,Description,Images,RecipeCategory,Keywords,RecipeIngredientQuantities,RecipeIngredientParts,AggregatedRating,ReviewCount,Calories,FatContent,SaturatedFatContent,CholesterolContent,SodiumContent,CarbohydrateContent,FiberContent,SugarContent,ProteinContent,RecipeServings,RecipeYield,RecipeInstructions
0,38.0,*38*,Low-Fat Berry Blue Frozen Dessert,1533.0,Dancer,PT24H,PT45M,PT24H45M,1999-08-09T21:46:00Z,Make and share this Low-Fat Berry Blue Frozen ...,"c(""https://img.sndimg.com/food/image/upload/w_...",Frozen Desserts,"c(""Dessert"", ""Low Protein"", ""Low Cholesterol"",...","c(""4"", ""1/4"", ""1"", ""1"")","c(""blueberries"", ""granulated sugar"", ""vanilla ...",4.5,4.0,170.9,2.5,1.3,8.0,29.8,37.1,3.6,30.2,3.2,4.0,NaN,"c(""Toss 2 cups berries with sugar."", ""Let stan..."
1,39.0,*39*,Biryani,1567.0,elly9812,PT25M,PT4H,PT4H25M,1999-08-29T13:12:00Z,Make and share this Biryani recipe from Food.com.,"c(""https://img.sndimg.com/food/image/upload/w_...",Chicken Breast,"c(""Chicken Thigh & Leg"", ""Chicken"", ""Poultry"",...","c(""1"", ""4"", ""2"", ""2"", ""8"", ""1/4"", ""8"", ""1/2"", ...","c(""saffron"", ""milk"", ""hot green chili peppers""...",3.0,1.0,1110.7,58.8,16.6,372.8,368.4,84.4,9.0,20.4,63.4,6.0,NaN,"c(""Soak saffron in warm milk for 5 minutes and..."
2,40.0,*40*,Best Lemonade,1566.0,Stephen Little,PT5M,PT30M,PT35M,1999-09-05T19:52:00Z,This is from one of my first Good House Keepi...,"c(""https://img.sndimg.com/food/image/upload/w_...",Beverages,"c(""Low Protein"", ""Low Cholesterol"", ""Healthy"",...","c(""1 1/2"", ""1"", NA, ""1 1/2"", NA, ""3/4"")","c(""sugar"", ""lemons, rind of"", ""lemon, zest of""...",4.5,10.0,311.1,0.2,0.0,0.0,1.8,81.5,0.4,77.2,0.3,4.0,NaN,"c(""Into a 1 quart Jar with tight fitting lid, ..."
3,41.0,*41*,Carina's Tofu-Vegetable Kebabs,1586.0,Cyclopz,PT20M,PT24H,PT24H20M,1999-09-03T14:54:00Z,This dish is best prepared a day in advance to...,"c(""https://img.sndimg.com/food/image/upload/w_...",Soy/Tofu,"c(""Beans"", ""Vegetable"", ""Low Cholesterol"", ""We...","c(""12"", ""1"", ""2"", ""1"", ""10"", ""1"", ""3"", ""2"", ""2...","c(""extra firm tofu"", ""eggplant"", ""zucchini"", ""...",4.5,2.0,536.1,24.0,3.8,0.0,1558.6,64.2,17.3,32.1,29.3,2.0,4 kebabs,"c(""Drain the tofu, carefully squeezing out exc..."
4,42.0,*42*,Cabbage Soup,1538.0,Duckie067,PT30M,PT20M,PT50M,1999-09-19T06:19:00Z,Make and share this Cabbage Soup recipe from F...,"""https://img.sndimg.com/food/image/upload/w_55...",Vegetable,"c(""Low Protein"", ""Vegan"", ""Low Cholesterol"", ""...","c(""46"", ""4"", ""1"", ""2"", ""1"")","c(""plain tomato juice"", ""cabbage"", ""onion"", ""c...",4.5,11.0,103.6,0.4,0.1,0.0,959.3,25.1,4.8,17.7,4.3,4.0,NaN,"c(""Mix everything together and bring to a boil..."


## Convert data types of columns used into lists. Import model and convert predicted output into list.

In [ ]:
import os
import re
import io
import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup
from PIL import Image
from tensorflow.keras.models import load_model  # This should now work
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
import joblib
from fuzzywuzzy import fuzz, process
from IPython.display import display, Image as IPImage, HTML

# Load ingredient model
ingredient_model = load_model('../../food_ingredients_model.h5')

# Load label encoders for transforming labels to and from one-hot representations
label_encoder = joblib.load('../../label_encoder.joblib')
one_hot_encoder = joblib.load('../../one_hot_encoder.joblib')

# Decodes image bytes into a normalized array for model input
def decode_image(image_bytes):
    img = Image.open(io.BytesIO(image_bytes)).convert('RGB')
    img = img.resize((128, 128))
    img_array = np.array(img, dtype=np.float32) / 255.0
    return img_array

# Predicts the ingredients in the given images
def predict_ingredients(image_bytes_list):
    predicted_ingredients = []
    for image_bytes in image_bytes_list:
        img = decode_image(image_bytes)
        img = np.expand_dims(img, axis=0)
        predictions = ingredient_model.predict(img)
        predicted_label = one_hot_encoder.inverse_transform(predictions)
        predicted_label = label_encoder.inverse_transform([int(predicted_label[0][0])])
        predicted_ingredients.extend(predicted_label)
    return predicted_ingredients

# Scrapes images for the given query from Google Images and saves them locally.
def download_images(query, num_images, output_folder="scraped_images"):

    search_url = f"https://www.google.com/search?q={query}&tbm=isch"
    headers = {
        "User-Agent": "Mozilla/5.0",
        "Referer": "https://www.google.com/"
    }
    
    response = requests.get(search_url, headers=headers)
    html = response.text
    
    soup = BeautifulSoup(html, 'html.parser')
    img_tags = soup.find_all('img')

    os.makedirs(output_folder, exist_ok=True)
    
    downloaded = 0
    for img_tag in img_tags:
        if downloaded >= num_images:
            break
        try:
            img_url = img_tag.get('src')
            if img_url is None:
                img_url = img_tag.get('data-src')
            if img_url:
                img_data = requests.get(img_url).content
                img = Image.open(io.BytesIO(img_data)).convert("RGB")
                img.save(os.path.join(output_folder, f"{query}_{downloaded+1}.jpg"))
                downloaded += 1
        except Exception as e:
            print(f"Error downloading image {downloaded+1} for {query}: {e}")
    
    print(f"Downloaded {downloaded} images for '{query}' into '{output_folder}'")


def load_images_from_folder(folder_path):
    image_bytes_list = []
    for filename in os.listdir(folder_path):
        if filename.endswith(('.png', '.jpg', '.jpeg', '.webp')):
            with open(os.path.join(folder_path, filename), 'rb') as f:
                image_bytes = f.read()
                image_bytes_list.append(image_bytes)
    return image_bytes_list

# List of ingredients to scrape images for
ingredient_queries = ['extra firm tofu', 'eggplant', 'zucchini', 'mushrooms', 'soy sauce', 
                      'olive oil', 'maple syrup', 'honey', 'red wine vinegar', 'lemon juice', 
                      'garlic cloves', 'mustard powder', 'black pepper']

# Download images for each ingredient
for ingredient in ingredient_queries:
    download_images(ingredient, num_images=20, output_folder="scraped_images")


imported_images = load_images_from_folder('scraped_images')

# Predict ingredients in the imported images
classified_ingredients = predict_ingredients(imported_images)

# Remove duplicates
classified_ingredients = list(set(classified_ingredients))  

print("Classified Ingredients:", classified_ingredients)

# Convert RecipeIngredientParts to lists using regex
def parse_ingredients(val):
    if isinstance(val, str):
        #find all quoted strings within c() notation
        matches = re.findall(r'"(.*?)"', val)
        return matches
    return val


# Convert ingredient to list
clean_recipe_df.loc[:, 'RecipeIngredientParts'] = clean_recipe_df['RecipeIngredientParts'].apply(parse_ingredients)
# Continue to convert other c() vectors to lists
clean_recipe_df.loc[:, 'RecipeIngredientQuantities'] = clean_recipe_df['RecipeIngredientQuantities'].apply(parse_ingredients)

# Convert recipe instructions and images to lists
clean_recipe_df.loc[:, 'RecipeInstructions'] = clean_recipe_df['RecipeInstructions'].apply(parse_ingredients)
clean_recipe_df.loc[:, 'Images'] = clean_recipe_df['Images'].apply(parse_ingredients)


#display(user_recipe_df[['Name', 'RecipeIngredientParts', 'RecipeIngredientQuantities', 'RecipeInstructions', 'Images']].head())

Error downloading image 1 for extra firm tofu: Invalid URL '/images/branding/searchlogo/1x/googlelogo_desk_heirloom_color_150x55dp.gif': No scheme supplied. Perhaps you meant https:///images/branding/searchlogo/1x/googlelogo_desk_heirloom_color_150x55dp.gif?
Downloaded 20 images for 'extra firm tofu' into 'scraped_images'
Error downloading image 1 for eggplant: Invalid URL '/images/branding/searchlogo/1x/googlelogo_desk_heirloom_color_150x55dp.gif': No scheme supplied. Perhaps you meant https:///images/branding/searchlogo/1x/googlelogo_desk_heirloom_color_150x55dp.gif?
Downloaded 20 images for 'eggplant' into 'scraped_images'
Error downloading image 1 for zucchini: Invalid URL '/images/branding/searchlogo/1x/googlelogo_desk_heirloom_color_150x55dp.gif': No scheme supplied. Perhaps you meant https:///images/branding/searchlogo/1x/googlelogo_desk_heirloom_color_150x55dp.gif?
Downloaded 20 images for 'zucchini' into 'scraped_images'
Error downloading image 1 for mushrooms: Invalid URL '/i

## Function to match model ouptut with recipe data (string matching, regex)

In [ ]:


def match_ingredients(recipe_ingredients, classified_ingredients):
    match_score = 0
    matched_ingredients = set()
    
    expanded_classified = set(classified_ingredients)

    
    # Exact and fuzzy matching
    for ingredient in expanded_classified:
        if ingredient in recipe_ingredients:
            match_score += 1
            matched_ingredients.add(ingredient)
        else:
            matches = process.extract(ingredient, recipe_ingredients, limit=1)
            if matches and matches[0][1] > 80:
                match_score += 1
                matched_ingredients.add(matches[0][0])
    
    return match_score, matched_ingredients
    
    return match_score, matched_ingredients

# Apply matching df
clean_recipe_df.loc[:, 'MatchScore'] = clean_recipe_df['RecipeIngredientParts'].apply(lambda x: match_ingredients(x, classified_ingredients)[0])
clean_recipe_df.loc[:, 'MatchedIngredients'] = clean_recipe_df['RecipeIngredientParts'].apply(lambda x: match_ingredients(x, classified_ingredients)[1])

# match scores
display(clean_recipe_df[['Name', 'RecipeIngredientParts', 'MatchScore', 'MatchedIngredients']])

## Function to display recipes (name, image, ingredients, instructions) with high string matching scores

In [ ]:

## TODO: output recipes with high matching scores, output their photo, name, ingredients, and instructions

#display recipes with high matching scores
def display_high_matching_recipes(df, score_threshold=5):
    high_matching_recipes = df[df['MatchScore'] >= score_threshold]
    
    for index, row in high_matching_recipes.iterrows():
        # Recipe name
        display(HTML(f"<h2>{row['Name']}</h2>"))
        
        # Recipe image
        if isinstance(row['Images'], list) and len(row['Images']) > 0:
            display(IPImage(url=row['Images'][0], width=400))
        
        # Ingredients
        display(HTML("<h3>Ingredients:</h3>"))
        ingredients_with_quantities = zip(row['RecipeIngredientParts'], row['RecipeIngredientQuantities'])
        ingredient_list_html = "<ul>"
        for ingredient, quantity in ingredients_with_quantities:
            if quantity:
                ingredient_list_html += f"<li>{ingredient}: {quantity}</li>"
            else:
                ingredient_list_html += f"<li>{ingredient}</li>"
        ingredient_list_html += "</ul>"
        display(HTML(ingredient_list_html))
        
        # Instructions
        display(HTML("<h3>Instructions:</h3>"))
        instructions = row['RecipeInstructions']
        instruction_list_html = "<ol>"
        for instruction in instructions:
            instruction_list_html += f"<li>{instruction.strip()}</li>"
        instruction_list_html += "</ol>"
        display(HTML(instruction_list_html))

# Display recipes with high match scores
display_high_matching_recipes(clean_recipe_df, score_threshold=9)